In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/03 22:55:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
rm -r nb/data

!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.zip
!wget https://pages.cs.wisc.edu/~harter/cs544/data/arid2017_to_lei_xref_csv.zip
!wget https://pages.cs.wisc.edu/~harter/cs544/data/code_sheets.zip
!mkdir -p nb/data
!unzip -o hdma-wi-2021.zip -d nb/data
!unzip -o arid2017_to_lei_xref_csv.zip -d nb/data
!unzip -o code_sheets.zip -d nb/data

In [5]:
!hdfs dfs -D dfs.replication=1 -cp -f data/*.csv hdfs://nn:9000/

In [6]:
banks_df = spark.read.csv("hdfs://nn:9000/arid2017_to_lei_xref_csv.csv")

In [9]:
banks_df.count()

5400

In [8]:
banks_df.rdd.take(5)

[Row(_c0='respondent_name', _c1='arid_2017', _c2='lei_2018', _c3='lei_2019', _c4='lei_2020'),
 Row(_c0='First National Bank', _c1='110004', _c2='5493003EW6T31TGECO83', _c3='5493003EW6T31TGECO83', _c4='5493003EW6T31TGECO83'),
 Row(_c0='First Mid Bank & Trust, National Association', _c1='110045', _c2='549300XOTES5TCS8T794', _c3='549300XOTES5TCS8T794', _c4='549300XOTES5TCS8T794'),
 Row(_c0='First Hope Bank, A National Banking Association', _c1='110118', _c2='5493003XLOX5FDT9R120', _c3='5493003XLOX5FDT9R120', _c4='5493003XLOX5FDT9R120'),
 Row(_c0='First National Bank of Waterloo', _c1='110180', _c2='5493002GOGIUXB84JB89', _c3='5493002GOGIUXB84JB89', _c4='5493002GOGIUXB84JB89')]

In [10]:
banks_df.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4
0,respondent_name,arid_2017,lei_2018,lei_2019,lei_2020
1,First National Bank,110004,5493003EW6T31TGECO83,5493003EW6T31TGECO83,5493003EW6T31TGECO83
2,"First Mid Bank & Trust, National Association",110045,549300XOTES5TCS8T794,549300XOTES5TCS8T794,549300XOTES5TCS8T794
3,"First Hope Bank, A National Banking Association",110118,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120
4,First National Bank of Waterloo,110180,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89


In [12]:
banks_df = (spark.read.format("csv").option("header", True).load("hdfs://nn:9000/arid2017_to_lei_xref_csv.csv"))

In [13]:
banks_df.limit(5).toPandas()

,respondent_name,arid_2017,lei_2018,lei_2019,lei_2020
0,First National Bank,110004,5493003EW6T31TGECO83,5493003EW6T31TGECO83,5493003EW6T31TGECO83
1,"First Mid Bank & Trust, National Association",110045,549300XOTES5TCS8T794,549300XOTES5TCS8T794,549300XOTES5TCS8T794
2,"First Hope Bank, A National Banking Association",110118,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120
3,First National Bank of Waterloo,110180,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89
4,Cumberland Valley National Bank & Trust Company,110254,549300OULGV8ZW0JV320,549300OULGV8ZW0JV320,549300OULGV8ZW0JV320


In [14]:
banks_df.dtypes

[('respondent_name', 'string'),
 ('arid_2017', 'string'),
 ('lei_2018', 'string'),
 ('lei_2019', 'string'),
 ('lei_2020', 'string')]

In [16]:
from pyspark.sql.functions import col, expr

banks_df.select(expr("`arid_2017`")).limit(3).toPandas()

,arid_2017
0,110004
1,110045
2,110118


In [18]:
banks_df.createTempView("banks")

In [19]:
spark.sql("SELECT * FROM banks LIMIT 3").toPandas()

,respondent_name,arid_2017,lei_2018,lei_2019,lei_2020
0,First National Bank,110004,5493003EW6T31TGECO83,5493003EW6T31TGECO83,5493003EW6T31TGECO83
1,"First Mid Bank & Trust, National Association",110045,549300XOTES5TCS8T794,549300XOTES5TCS8T794,549300XOTES5TCS8T794
2,"First Hope Bank, A National Banking Association",110118,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120


In [20]:
spark.sql("SHOW TABLES").show()

23/11/03 23:21:27 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |    banks|       true|
+---------+---------+-----------+



In [21]:
spark.sql("""
SELECT *
FROM banks
WHERE respondent_name LIKE "First%"
LIMIT 10
""").toPandas()

,respondent_name,arid_2017,lei_2018,lei_2019,lei_2020
0,First National Bank,110004,5493003EW6T31TGECO83,5493003EW6T31TGECO83,5493003EW6T31TGECO83
1,"First Mid Bank & Trust, National Association",110045,549300XOTES5TCS8T794,549300XOTES5TCS8T794,549300XOTES5TCS8T794
2,"First Hope Bank, A National Banking Association",110118,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120
3,First National Bank of Waterloo,110180,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89
4,First National Bank of North Arkansas,110406,5493000LJB0L73TDJT41,5493000LJB0L73TDJT41,5493000LJB0L73TDJT41
5,"First National Bank, Ames, Iowa",110408,549300OIJZOV6CPR1Y57,549300OIJZOV6CPR1Y57,549300OIJZOV6CPR1Y57
6,First National Bank,110465,54930035C68SRQ1RS395,54930035C68SRQ1RS395,54930035C68SRQ1RS395
7,First National Bank of South Carolina,110680,549300X0240UXBEJNV56,549300X0240UXBEJNV56,None
8,First National Bank Of Oklahoma,111397,549300OBYIDGN0LRQS38,549300OBYIDGN0LRQS38,549300OBYIDGN0LRQS38
9,First National Community Bank,111412,549300Q26XXW77S35W27,549300Q26XXW77S35W27,549300Q26XXW77S35W27
